In [1]:
import sys
from pathlib import Path
import os
# Add project root to sys.path
sys.path.append(str(Path().resolve().parent))  # Adjust if needed

import numpy as np
import pandas as pd

from generators.full_generators import (
    create_company_data
)

company_name = "Vestas"
count_products = 10
count_employees = 10

data = create_company_data(company_name, count_employee = count_employees, count_products = count_products, count_accounts = 30, count_departments = 10, count_customers = 10, save_to_csv=True)

Generating data for company: Vestas...
✔ Roles and Names generated.
✔ Payroll data generated.
✔ Procurement data generated.
✔ Services data generated.
✔ Products data generated.
✔ Accounts, Customers, and Departments generated.
✔ All CSVs saved to: data/outputdata/
✔ All mapping data generated.
✔ All mapping CSVs saved to: data/outputdata/mapping
✔ All erp-data generated.
✔ All ERP CSVs saved to: data/outputdata/fact
✔ All ERP data and mapping generated for company: Vestas
